# Behavior of users of the mobile app based on selling groceries

1. [Importing libraries and dataset](#start)   
2. [Data preprocessing](#prep)
    * [Summary](#sum_prep)
3. [Exploratory data analysis](#eda)
    * [How many events in logs](#events_in_log)
    * [How many users in log?](#users_in_log)
    * [How many on average events is visited by one unique user?](#mean_events_per_user_in_log)
    * [Time period in logs](#time_period_in_log)
    * [The number of events and users in the updated data](#events_and_users_in_update_log)
    * [Distribution of users by experiment groups in the updated data](#test_groups_in_update_log)
    * [Summary](#sum_analysis)
4. [Purchase (customer) funnel](#customer_funnel)
    * [Events in log](#events_in_log)
    * [The percentage of users who committed each event](#events_per_user)
    * [Funnel of events](#funnel)
    * [Summary](#sum_customer_funnel)
5. [Results of the A/A/B-test experiment](#aab_test)
    * [Distribution by experimental groups](#groups_in_aab_test)
    * [A/A-test](#aa_test)
    * [Distribution of users groups by events](#comparison_aa_test)
    * [A/A/B-test: results](#results_aab_test)
    * [A/A/B-test: with different alpha level](#p_values_aab_test)
    * [Summary](#sum_aab_test)
6. [Conclusion](#conclusion)

<a id="start"></a>
## Importing libraries and dataset

Importing the necessary libraries for further work

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import scipy.stats as stats
from matplotlib import pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
import matplotlib
import math

In [2]:
%config InlineBackend.figure_format = 'retina'
matplotlib.style.use('ggplot')

Let's read the existing `csv` and create a data frame - `df` 

Let's output the first five rows

In [3]:
df = pd.read_csv('logs_exp.csv', delimiter ='\t')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'logs_exp.csv'

Now we're ready to start preprocessing our data

<a id="prep"></a>
## Data preprocessing

Rename columns for future

In [ ]:
df.columns = ['event_name', 'device_id_hash', 'event_timestamp', 'experiment_id']

### Description of data
Each log entry is a user action or event.
* `event_name` — event name;
* `device_id_hash` — unique user ID;
* `event_timestamp` — time of the event;
* `experinet_id` — experiment number: **246** and **247** — control groups, and **248** — experimental

Let's add new columns: change type to datetime - `date_time`, and second with only data - `date`

In [ ]:
df['date_time'] = pd.to_datetime(df['event_timestamp'], unit='s')
df['date'] = df['date_time'].dt.date
df.head()

Let's check the nans and right data types of columns and output information about the dataframe.

In [ ]:
df.info()

Remove duplicates

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates()
df.head()

<a id="sum_prep"></a>
### Summary

In this section we:
* renamed columns for more convenient operation
* remove rows with obvious duplicates
* created two columns with datetime type of values and with date

In general, we processed the data for further work.

<a id="eda"></a>
## Exploratory data analysis

<a id="events_in_log"></a>
### How many events in logs?

In [ ]:
len(df['event_name'])

Unique events in log 5 - `MainScreenAppear`, `OffersScreenAppear`, `CartScreenAppear`, `PaymentScreenSuccessful`, `Tutorial`

In [ ]:
df['event_name'].nunique()

<a id="users_in_log"></a>
### How many users in log?

It's 7551 unique users in dataset

In [ ]:
df['device_id_hash'].nunique()

<a id="mean_events_per_user_in_log"></a>
### How many on average events is visited by one unique user?

In [ ]:
table = pd.pivot_table(df, values='device_id_hash', index=['event_name'], aggfunc=['nunique', 'count'])
table.columns=['unique_users', 'number_of_events']
table['mean_events_per_user'] = round(table['number_of_events']/table['unique_users'], 2)
table.sort_values(by='number_of_events', ascending=False)

In [ ]:
table = table.reset_index().sort_values(by='number_of_events', ascending=False)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=table['event_name'],
    y=table['mean_events_per_user'],
    marker_color='#F2B680', 
    text=table['mean_events_per_user']
))
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_layout(title_text='Average number of events per user',title_y=0.9, title_x=0.5)
fig.show()

<a id="time_period_in_log"></a>
### Time period in logs

Let's find tha max and the min in time period of logs

In [ ]:
print('Min date:', df['date'].min())
print('Max date:', df['date'].max())

Plot histogram

In [ ]:
from scipy import stats
def freedman_diaconis(data, returnas="width"):
    '''Count right number of bins'''
    data = np.asarray(data, dtype=np.float_)
    IQR = stats.iqr(data, rng=(25, 75), scale=1.0, nan_policy="omit")
    N = data.size
    bw = (2 * IQR) / np.power(N, 1/3)

    if returnas=="width":
        result = bw
    else:
        datmin, datmax = data.min(), data.max()
        datrng = datmax - datmin
        result = int((datrng / bw) + 1)
    return(result)

In [ ]:
NBR_BINS = freedman_diaconis(df['date_time'], returnas="bins")

fig, ax = plt.subplots(nrows=1, ncols=1)
fig.set_size_inches(13, 8)
ax.hist(df['date_time'], NBR_BINS, color= '#F2B680', edgecolor="black", linewidth=1.1)
ax.grid(False)
ax.set_xlabel('Date', fontsize = 15)
ax.set_ylabel('Number of events', fontsize = 15)
ax.set_title('Distribution of the number of events by time', fontsize = 17)
plt.tight_layout()
plt.show()

We can see that we do not have the same complete data for the entire period, since technically, in login days, events from the past "arrive" for some users - this shifted the data. 

On August 1, 2019, the data became complete, and we can discard the data before that date.

In [ ]:
df_update = df.query("date_time >= '2019-08-01'")

It turns out that in reality we have data for the period - from 2019-08-01 to 2019-08-07

In [ ]:
print('Min date:', df_update['date'].min())
print('Max date:', df_update['date'].max())

<a id="events_and_users_in_update_log"></a>
### The number of events and users in the updated data

We lost less than 1% of users and less than 2% of events.

In [ ]:
x = df['device_id_hash'].nunique() - df_update['device_id_hash'].nunique()
print(f'We removed {x} users')
print("It's", round(x/df['device_id_hash'].nunique()*100.0, 2), '%')

print('')

x = len(df['event_name']) - len(df_update['event_name'])
print(f'We removed {x} logs')
print("It's", round(x/len(df['event_name'])*100.0, 2), '%')

<a id="test_groups_in_update_log"></a>
### Distribution of users by experiment groups in the updated data

Let's check that we have users from all three experimental groups

According to our updated data, all groups are represented, while each group accounts for approximately 33% of all users in the log

In [ ]:
table = pd.pivot_table(df_update, values='device_id_hash', index=['experiment_id'], aggfunc='nunique')

table

In [ ]:
table = table.reset_index()
colors = ['#F3E79B', '#FAC484', '#F8A07E']
fig = px.pie(table, values='device_id_hash', names='experiment_id', color_discrete_sequence=px.colors.sequential.Sunset)
fig.update_layout(
    legend=dict(
        x=1,
        y=1,
        traceorder="reversed",
        font=dict(
            family="Arial",
            size=17,
            color="#361812"
        ),
        bgcolor="LightGrey",
        bordercolor="Black",
        borderwidth=0.5))
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.update_traces(hoverinfo='label+value+percent', textinfo='percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#361812', width=2)))
fig.update_layout(title_text='Distribution of users by experiment groups in the updated data',title_y=0.95, title_x=0.5)
fig.show()

<a id="sum_analysis"></a>
### Summary

In this section we:
* Studied the distribution of data by date and time
* Created a dataframe with updated data, cleaned the data from irrelevant (with an "outdated date")

We will continue the analysis with data for the period from 2019-08-01 to 2019-08-07. In the updated data, each group of the experiment is 33%

<a id="customer_funnel"></a>
## Purchase (customer) funnel

<a id="events_in_log"></a>
### Events in log

Let's see what events are in the logs, how often they occur and sort them by frequency.

The most common event is `MainScreenAppear`, that is, the opening of the main screen

In [ ]:
table = df_update['event_name'].value_counts().to_frame().reset_index()
table.columns = ['event_name', 'number_of_events']
table

<a id="events_per_user"></a>
### The percentage of users who committed each event

Let's count how many users committed each of these events and sort them by the number of users. 

We will also calculate the proportion of users who have ever committed an event.

In [ ]:
table = pd.pivot_table(df_update, values='device_id_hash', index=['event_name'], aggfunc=['nunique', 'count'])
table.columns=['unique_users', 'number_of_events']
table['part_of_all_users'] = round(table['unique_users']/df_update['device_id_hash'].nunique()*100, 2)
table.sort_values(by='number_of_events', ascending=False)

<a id="funnel"></a>
### Funnel of events

According to the table, the funnel can be represented as:
1. `MainScreenAppear` (main screen) ->
2. `OffersScreenAppear` (offers screen) ->
3. `CartScreenAppear` (shopping cart screen) ->
4. `PaymentScreenSuccessful` (successful payment screen)

Much fewer users visited the `Tutorial` stage than at other stages (11% of users visited). We will not take this stage into account when calculating the funnel.

Not absolutely all users get to the `MainScreenAppear` event (only 98.5%), which may, for example, indicate that there are other ways to "walk through the funnel`

In [ ]:
df_update = df_update.query('event_name != "Tutorial"')

In [ ]:
table = table.reset_index().sort_values(by='number_of_events', ascending=False)
table = table[:4]

Let's build a funnel graph

In [ ]:
fig = go.Figure(
    go.Funnel(
    y=table['event_name'],
    x=table['unique_users'],
    textposition = "inside",
    textinfo = "value+percent initial+percent previous",
    opacity = 0.75, marker = {"color": ["#F3E79B", "#FAC484", "#F8A07E", "#F56D53"],
    "line": {"width": [4, 2, 2, 3, 1, 1], "color": ["#361812", "#3792A6", "#361812", "#361812"]}},
    textfont = {'family': "sans-serif", 'size': [16, 16, 16, 16], 'color': ["#361812", "#361812", "#361812", "#361812"]},
    connector = {"line": {"color": "#3792A6", "dash": "dot", "width": 4}})
    )
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.update_layout(title_text='Воронка продаж, процент от числа пользователей на первом этапе', title_x=0.5)
fig.show() 

**What percentage of users goes to the next step of the funnel (from the number of users on the previous one)?**

In [ ]:
funnel = table[['event_name', 'unique_users']].set_index('event_name')
funnel = funnel.pct_change()
funnel['unique_users'] = round(funnel['unique_users'], 2)
funnel.columns = ['share_users_next_step']
funnel.fillna(1)

**At which step do we lose the most users?**
* It can be seen that when switching from the main screen (`Main Screen Appears`) to the offers screen (`OffersScreenAppear`), almost **40%** of users "leave"

**What percentage of users reaches from the first event to the payment?**

* According to the funnel graph, we see that almost half of the users "reach" the payment (`PaymentScreenSuccessful`) from the first step - **48%** of the number of users who were on the main screen (`MainScreenAppear`)

<a id="sum_customer_funnel"></a>
### Summary

We found out:
* **48%** of users from the number of those who were on the home screen reach the purchase
* When switching from the main screen (`Main Screen Appears`) to the offers screen (`OffersScreenAppear`), "leaves" **38%** of users
* Almost **20% does not go from the offer screen (`OffersScreenAppear`) to the cart (`CartScreenAppear`)**
* From the shopping cart screen (`CartScreenAppear`) successfully pay (`PaymentScreenSuccessful`) **95%** of users

<a id="aab_test"></a>
## Results of the A/A/B-test experiment

<a id="groups_in_aab_test"></a>
### Distribution by experimental groups
How many users are in each experimental group?

The table shows the distribution of unique users by groups

In [ ]:
table = pd.pivot_table(df_update, values='device_id_hash', index=['experiment_id'], aggfunc=['nunique'])
table.columns = ['number_unique_users']
table = table.reset_index()
table

<a id="aa_test"></a>
### A/A-test
We have 2 control groups for an A/A experiment to check the correctness of all mechanisms and calculations. 

Let's check whether statistical criteria find the difference between samples `246` and `247'.

In [ ]:
# form dataframes with data on events and the number of users for each group
group_246 = (
    df_update[df_update['experiment_id'] == 246]
    .groupby('device_id_hash', as_index=False)
    .agg({'event_name': pd.Series.nunique})
)
group_246.columns = ['user', 'number_of_events']

group_247 = (
    df_update[df_update['experiment_id'] == 247]
    .groupby('device_id_hash', as_index=False)
    .agg({'event_name': pd.Series.nunique})
)
group_247.columns = ['user', 'number_of_events'] 

Let's check the sample means, they should be normally distributed

According to the graph, we see that the distribution in both groups is close to normal, this is also shown by the Shapiro-Wilk test. Let's continue the analysis.

In [ ]:
# dividing our data into groups so that each value falls into some kind of sample
group_246['bucket'] = group_246['user'].apply(lambda x: hash(x) % 100)
group_247['bucket'] = group_247['user'].apply(lambda x: hash(x) % 100)
# group by the new values and calculate the mean
bucket_on = group_246.groupby('bucket')[['number_of_events']].mean()
bucket_off = group_247.groupby('bucket')[['number_of_events']].mean()
# probability plot
plt.figure(figsize=(14, 6))
plt.subplot(2,2,1)
stats.probplot(bucket_on['number_of_events'], dist='norm', plot=plt)
plt.subplot(2,2,2)
stats.probplot(bucket_off['number_of_events'], dist='norm', plot=plt)
plt.show()

In [ ]:
# Shapiro-Wilk test
sp = stats.shapiro(bucket_on['number_of_events'])
concl = '=> not statistically significant, normal distribution' if sp[1]>0.05 else '=> statistically significant, not a normal distribution' 
print('Test for normal distibution for 246 group: W = %.3f' % sp[0], ', p-value = %.3f' % sp[1], concl)

sp = stats.shapiro(bucket_off['number_of_events'])
concl = '=> not statistically significant, normal distribution' if sp[1]>0.05 else '=> statistically significant, not a normal distribution' 
print('Test for normal distibution for 247 group: W = %.3f' % sp[0], ', p-value = %.3f' % sp[1], concl)

Let's check the static significance of hypotheses
* H0 - the mean number of events for visitors from groups 246 and 247 is equal to
* H1 - the mean number of events for visitors from groups 246 and 247 is not equal

Based on the data obtained, we accept the null hypothesis

In [ ]:
alpha = 0.05

for column in ['number_of_events']:
    print('For column -', column)
    print('Means:', round(group_246[column].mean(),3), 'vs', round(group_247[column].mean(), 3))
    print('t-test')
    stat, pvalue = stats.ttest_ind(group_246[column], group_247[column])
    print('p-value =', round(pvalue,3))
    print('Rejecting H0' if alpha > pvalue / 2 else 'Accepting H0')
    print()
    
    print('Mann Witneyu-test')
    stat, pvalue = stats.mannwhitneyu(group_246[column], group_247[column], alternative='greater')
    print('p-value =', round(pvalue,3))
    print('Rejecting H0' if alpha > pvalue else 'Accepting H0')
    print()

According to the obtained data of the A/A test, the groups do not differ, that is, we made sure that in both groups the target metrics do not have a statistically significant difference. Let's continue the analysis.

<a id="comparison_aa_test"></a>
### Distribution of users groups by events
Let's calculate the number of users who committed each event in each of the control groups, and the proportion of users who committed this event

Let's create a function for this

In [ ]:
def groups_by_event(groups, events):
    user_all = []
    user_event = []
    events_tab = []
    groups_tab = []
    for event in events:
        for group in groups:
            t_1 = df_update[(df_update['event_name'] == event) & (df_update['experiment_id'] == group)]
            b = df_update[df_update['experiment_id'] == group].agg({'device_id_hash':'nunique'})
            a = t_1.agg({'device_id_hash':'nunique'})
            user_event.append(a[0])
            events_tab.append(event)
            groups_tab.append(group)
            if len(user_all) == 0:
                user_all.append(b[0])
            else:
                user_all.append(user_event[-2])
    d = {'group': groups_tab,
         'event_name':events_tab, 
         'users_in_all_group':user_all, 
         'users_on_event':user_event}
    data = pd.DataFrame(data=d)
    data = data.sort_values(by='group').reset_index().drop(columns=['index'])
    data['share'] = round(data['users_on_event']/data['users_in_all_group']*100,1)
    return data

In [ ]:
list_of_events = ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful']
group_246 = groups_by_event([246], list_of_events)
group_247 = groups_by_event([247], list_of_events)
group_247

Let's create a function to check whether the difference between groups is statistically significant using z-score

Let's check the static significance of hypotheses
* H0 - the number of visitors from groups 246 and 247 at certain events is equal to

* H1 - the number of visitors from groups 246 and 247 at certain events is not equal

Based on the data obtained, we accept the null hypothesis

In [ ]:
def testing_z_score(group_1, group_2, alpha):
    print('Comparing groups:',group_1.loc[0][0],'&', group_2.loc[0][0])
    print('------------------')
    for i in range(len(group_1)):
        print(group_1.loc[i][3])
        print(group_1.loc[i][2])
        p1 = group_1.loc[i][3]/group_1.loc[i][2]
        print(group_2.loc[i][3])
        print(group_2.loc[i][2])
        p2 = group_2.loc[i][3]/group_2.loc[i][2]
        p1_2 = (group_1.loc[i][3] + group_2.loc[i][3]) / (group_1.loc[i][2] + group_2.loc[i][2])
        difference = p2 - p1
        z_score = (difference / math.sqrt(p1_2*(1 - p1_2)
                                    * (1/group_1.loc[i][2] + 1/group_2.loc[i][2])))
        p_value = round(stats.norm.sf(abs(z_score))*2,5)
        print('Event:', group_1.loc[i][1])
        print('р-value:', p_value)
        if (p_value < alpha):
            print("Significant difference")
        else: 
            print("No difference")
        print('------------------')
            
testing_z_score(group_246, group_247, alpha)            

**Is it possible to say that the splitting into groups works correctly?**

Yes, there is no statistically significant difference in all events in the control groups

<a id="results_aab_test"></a>
### A/A/B-test: results
We will do the same with the group with the changed font (`248`). Let's compare the results with each of the control groups separately for each event.

**H0** - there is no difference between the number of users in the control and experimental group at a certain event

**H1** - there is a difference between the number of users in the control and experimental group at a certain event

In [ ]:
group_248 = groups_by_event([248], list_of_events)
group_248

In [ ]:
testing_z_score(group_246, group_248, alpha)

In [ ]:
testing_z_score(group_247, group_248, alpha)

In [ ]:
print('Number of control group users on PaymentScreenSuccessful event -', int(group_247[group_247['event_name'] == 'PaymentScreenSuccessful']['users_on_event']))
print('Number of experimental group users on PaymentScreenSuccessful event -', int(group_248[group_248['event_name'] == 'PaymentScreenSuccessful']['users_on_event']))
print('Difference:', int(group_248[group_248['event_name'] == 'PaymentScreenSuccessful']['users_on_event']) - int(group_247[group_247['event_name'] == 'PaymentScreenSuccessful']['users_on_event']), 'users')

As we can see from the test results, a significant difference is observed only between the groups `247` and `248` at the event `PaymentScreenSuccessful`, that is, we can say that the font change led to a statistically significant increase in the number of users who switched from the shopping cart stage to payment.

 **Let's compare the results with the combined control group**

**H0** - there is no difference between the number of users in the combined control and experimental group at a certain event

**H1** - there is a difference between the number of users in the combined control and experimental group at a certain event

Let's create a dataframe with data for both control groups

In [ ]:
group_493 = group_246+group_247
group_493['event_name'] = group_246['event_name']
group_493['share'] = round(group_493['users_on_event']/group_493['users_in_all_group']*100, 1)
group_493

In [ ]:
testing_z_score(group_493, group_248, alpha)

In [ ]:
print('The percentage of those who transferred from the basket to the payment in the control group -', int(group_493[group_493['event_name'] == 'PaymentScreenSuccessful']['share']),'%')
print('The percentage of those who transferred from the basket to the payment in the experimental group -', int(group_248[group_248['event_name'] == 'PaymentScreenSuccessful']['share']),'%')
print('Difference:', int(group_248[group_248['event_name'] == 'PaymentScreenSuccessful']['share']) - int(group_493[group_493['event_name'] == 'PaymentScreenSuccessful']['share']), '%')

Here we also see a statistically significant difference in the increase in the number of people who switched from `CartScreenAppear` to `PaymentScreenSuccessful`

**What conclusions can be drawn from the experiment?**

* With the significance stat level equal to 0.05, we observe a significant difference between groups 247 (control) and 248 (experimental) for the transition from the `CartScreenAppear` event to `PaymentScreenSuccessful` (93.5% in the control group, 96% in the experimental group)
* A significant difference also between the combined control groups 246 and 247 and the experimental group 248 on the same transition from shopping cart to payment (94% of the percentage in the control group passed, 96% in the experimental group)

<a id="p_values_aab_test"></a>
### A/A/B-test: with different alpha level
When testing the statistical hypotheses above, we used the frequently occurring significance stat level of 0.05.

Let's look at the test results at other levels of stat significance: at 0.01 and at 0.1.

We will conduct a test with statistical significance **`alpha`=0.01**

**H0** - there is no difference between the number of users in the control and experimental group at a certain event

**H1** - there is a difference between the number of users in the control and experimental group at a certain event

In [ ]:
alpha = .01

In [ ]:
testing_z_score(group_246, group_248, alpha)

In [ ]:
testing_z_score(group_247, group_248, alpha)

**H0** - there is no difference between the number of users in the combined control and experimental group at a certain event

**H1** - there is a difference between the number of users in the combined control and experimental group at a certain event

In [ ]:
testing_z_score(group_493, group_248, alpha)

At the level of significance in **0.01** that is, with a decrease in the probability of a first-kind error, only the difference remains statistically significant when comparing the transitions **from the  to the payment** between the control group `247` and the experimental `248`.

Let's look at the results with statistical significance **`alpha`=0.10**

**H0** - there is no difference between the number of users in the control and experimental group at a certain event

**H1** - there is a difference between the number of users in the control and experimental group at a certain event

In [ ]:
alpha = .1

In [ ]:
testing_z_score(group_246, group_248, alpha)

In [ ]:
testing_z_score(group_247, group_248, alpha)

**H0** - there is no difference between the number of users in the combined control and experimental group at a certain event

**H1** - there is a difference between the number of users in the combined control and experimental group at a certain event

In [ ]:
testing_z_score(group_493, group_248, alpha)

With an increase in the probability of an error of the first kind, the same differences remain statistically significant as with *alpha=0.05*

To avoid the problem of multiple comparisons, we apply the Bonferroni correction

In [ ]:
def testing_z_score_pvalue(group_1, group_2):
    pvalue = []
    for i in range(len(group_1)):
        p1 = group_1.loc[i][3]/group_1.loc[i][2]
        p2 = group_2.loc[i][3]/group_2.loc[i][2]
        p1_2 = (group_1.loc[i][3] + group_2.loc[i][3]) / (group_1.loc[i][2] + group_2.loc[i][2])
        difference = p2 - p1
        z_score = (difference / math.sqrt(p1_2*(1 - p1_2)
                                    * (1/group_1.loc[i][2] + 1/group_2.loc[i][2])))
        p_value = round(stats.norm.sf(abs(z_score))*2,5)
        if p_value > .05:
            pvalue.append(p_value)
    return pvalue

gr246_248 = testing_z_score_pvalue(group_246, group_248)
gr247_248 = testing_z_score_pvalue(group_247, group_248)
gr439_248 = testing_z_score_pvalue(group_493, group_248)

raw_pvalue = gr246_248 + gr247_248 + gr439_248

In [ ]:
from statsmodels.stats.multitest import multipletests

alpha = .05

rejected, p_adjusted, _, alpha_corrected = multipletests(raw_pvalue, alpha=alpha, 
                               method='bonferroni', is_sorted=False, returnsorted=False)
print(np.sum(rejected))
print(alpha_corrected)

Based on the results of the Bonferroni correction, we leave all confirmed alternative hypotheses.

<a id="sum_aab_test"></a>
### Summary

Using the results of the **A/A/B test**, we wanted to find out **will be new font better for the app**. 

The designers wanted to change the fonts throughout the application, but we came to the conclusion that there is no statistically significant difference in all transitions through the sales funnel. 

As we can see from the tests:
* with a statistical significance of 0.01 (99% probability of error of the first kind) - a significant difference between the control (247) and experimental (248) groups for the transition from the event `CartScreenAppear` to `PaymentScreenSuccessful`

    **in the control group, 93.5% switched from the shopping basket to payment, in the experimental group - 96%**
    
    
* with alpha=0.05 - a significant difference remains the significance of the difference of those who moved from the basket to payment, and also the difference between the combined two control group and the experimental group is significant with the same transition from `CartScreenAppear` to `PaymentScreenSuccessful`
    
    **in the control group, 94% percent passed, in the experimental group - 96%**
    
    
* if the probability of a first-kind error is 90% (stat significance level 0.1) - stat, the difference between groups 247 (control) and 248 (experimental) and the combined control group and 248 is significant when switching from the shopping cart to the successful payment screen

<a id="conclusion"></a>
## Conclusion

Startup sells food. We wanted to find out how the users of our mobile application behave.

We studied the sales (customer) funnel and came to the following conclusions:
1. The sales funnel is a chain of events: 1) `MainScreenAppear` (main screen) -> 
2) `OffersScreenAppear` (offers screen) -> 3) `CartScreenAppear` (shopping cart) -> 4) `PaymentScreenSuccessful` (successful payment screen)
2. The purchase reaches **48%** of users from the number of those who were on the home screen
3. When switching from the main screen (`MainScreenAppear`) to the offers screen (`OffersScreenAppear`), **38%** of users "leave"
4. Almost **20% does not go from the offer screen (`OffersScreenAppear`) to the shopping cart (`CartScreenAppear`)**
5. From the cart(`CartScreenAppear`) successfully pay (`PaymentScreenSuccessful`) **95%** of users

According to the results of the A/A/B experiment, this conclusion was obtained:
1. There is no need to change the fonr in the entire app, a significant difference in results was observed only between the control (247) and experimental (248) groups for the transition from the CartScreenAppear event to the PaymentScreenSuccessful event (successful payment) and between the combined of the two control groups and the experimental group with the same transition from `CartScreenAppear` to `PaymentScreenSuccessful`

**in the control group, 94% switched from the shopping basket to payment, in the experimental group - 96%**

It is worth paying attention to the transition from `MainScreenAppear` (main screen) to `OffersScreenAppear` (offers screen).

## 